### 6.1 CNN/DailyMail 데이터셋

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", version="3.0.0")  # 익명화 처리 안된 버전
print(f"특성: {dataset['train'].column_names}")

In [ ]:
sample = dataset["train"][1]
print(f"""기사 (500개 문자 발췌, 총 길이: {len(sample["article"])}):""")
print(sample['article'][:500])
print(f'\n요약 (길이: {len(sample["highlights"])}):')
print(sample["highlights"])

### 6.2 텍스트 요약 파이프라인

In [ ]:
sample_text = dataset["train"][1]["article"][:2000]
# 딕셔너리에 각 모델이 생성한 요약을 저장합니다.
summaries = {}

In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

#### 6.2.1 요약 기준 모델

In [ ]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

summaries['baseline'] = three_sentence_summary(sample_text)

In [ ]:
print(summaries)

#### 6.2.2 GPT-2

In [ ]:
from transformers import pipeline, set_seed

set_seed(42)

pipe = pipeline("text-generation", model="gpt2-xl")
gpt2_query = sample_text + "\nTL:DR:\n"  # gpt에서 요약을 얻기 위한 query
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(
    sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))

#### 6.2.3 T5

In [ ]:
pipe = pipeline("summarization", model="t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

#### 6.2.4 BART

In [ ]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

#### 6.2.5 PEGASUS

In [4]:
!pip install sentencepiece
!pip install protobuf


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.3/400.3 kB 5.6 MB/s eta 0:00:00 MB/s eta 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")

# pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")
# pipe_out = pipe(sample_text)
# summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates